In [6]:
# Cell 1: Imports
import numpy as np
import matplotlib.pyplot as plt
import imageio
from matplotlib import cm

np.random.seed(42)

In [11]:
# Cell 2: Parameters and Data
N = 3100               # Number of data points
M = 31                 # Number of clusters
beta_min = 0.1
beta_max = 100.0
tau = 1.05

#X = np.random.randn(N, 2)               # 2D input data
X = np.loadtxt("D31.txt")[:,:2]
rho = np.ones(N) / N                    # Uniform weights

# Initialize cluster centers randomly
Y = np.random.randn(M, 2)

# Random probabilities p(l | j, i): shape (M, M, N)
p_l_given_ji = np.zeros((M, M, N))  # l, j, i
for j in range(M):
    for i in range(N):
        p_l_given_ji[:, j, i] = np.random.dirichlet(np.ones(M))

frames = []

In [12]:
# Cell 3: Distance function
def d(x, y):
    return np.linalg.norm(x - y) ** 2

In [ ]:
# Cell 4: Main loop with policy and y_l updates
beta = beta_min

while beta <= beta_max:
    converged = False
    while not converged:
        # Step 1: Compute π(y_j | x_i)
        pi = np.zeros((N, M))  # shape: (num_data_points, num_clusters)
        
        for i in range(N):
            numerators = np.zeros(M)
            for j in range(M):
                total = 0.0
                for l in range(M):
                    total += p_l_given_ji[l, j, i] * d(X[i], Y[l])
                numerators[j] = np.exp(-beta * total)
        
            denom = np.sum(numerators)
            if denom == 0:
                pi[i] = np.ones(M) / M  # fallback to uniform if all numerators were zero
            else:
                pi[i] = numerators / denom

        # Step 2: Update cluster centers y_l
        Y_new = np.zeros_like(Y)
        for l in range(M):
            numerator = np.zeros(2)
            denominator = 0
            for i in range(N):
                for j in range(M):
                    pij = pi[i][j]
                    plji = p_l_given_ji[l,j,i]
                    numerator += rho[i] * pij * plji * X[i]
                    denominator += rho[i] * pij * plji
            if denominator > 0:
                Y_new[l] = numerator / denominator
            else:
                Y_new[l] = Y[l]

        # Check for convergence
        if np.allclose(Y, Y_new, atol=1e-3):
            converged = True
        Y = Y_new

    # Plot for GIF
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(X[:, 0], X[:, 1], alpha=0.5, label="Data")
    ax.scatter(Y[:, 0], Y[:, 1], color='red', s=100, marker='X', label="Centers")
    ax.set_title(f"β = {beta:.2f}")
    ax.legend()
    ax.grid(True)

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frames.append(image)
    plt.close()

    beta *= tau

In [ ]:
# Cell 5: Save animation
imageio.mimsave("phase_transition_clustering.gif", frames, fps=2)
print("GIF saved as 'phase_transition_clustering.gif'")